In [1]:
import io
import itertools

import numpy as np
import sklearn.metrics
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

In [2]:
#loading the data
train_data = np.load(r'datasets/Trousers & Jeans - All - Train.npz')
val_data = np.load(r'datasets/Trousers & Jeans - All - Validation.npz')
test_data = np.load(r'datasets/Trousers & Jeans - All - Test.npz')

In [3]:
train_images, train_labels = train_data['images'], train_data['labels']
val_images, val_labels = val_data['images'], val_data['labels']
test_images, test_labels = test_data['images'], test_data['labels']

In [4]:
#feature scaling
train_images = train_images/255.
val_images = val_images/255.
test_images = test_images/255.

In [5]:
#constant definition
NUM_EPOCHS = 15
BATCH_SIZE = 64

In [7]:
#hyperparam definition
HP_FILTER_SIZE = hp.HParam('filter_size', hp.Discrete([3,5,7]))
HP_NUM_FILTER = hp.HParam('filters_number', hp.Discrete([32,64,96,128]))
HP_FILTER_SIZE_2 = hp.HParam('filter_size_2', hp.Discrete([3,5]))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer(r'Logs/Model 3/hparam_tuning/').as_default():
    hp.hparams_config(
        hparams= [HP_FILTER_SIZE,HP_NUM_FILTER],
        metrics = [hp.Metric(METRIC_ACCURACY,display_name='accuracy')]
    )

In [14]:
#model definition and training
def train_test_model(hparams,session_num):
    
    #model outline
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(hparams[HP_NUM_FILTER],hparams[HP_FILTER_SIZE], activation='relu', input_shape=(120,90,3)),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(hparams[HP_NUM_FILTER],hparams[HP_FILTER_SIZE_2],activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(4)
    ])
    
    
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    
    model.compile(optimizer='adam', loss= loss_fn, metrics=['accuracy'])
    
    #defining the logging dir
    log_dir = 'Logs\\Model 3\\fit\\'+'run-{}'.format(session_num)
    
    def plot_confusion_matrix(cm, class_names):
        figure = plt.figure(figsize=(12, 12))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title("Confusion matrix")
        plt.colorbar()
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names, rotation=45)
        plt.yticks(tick_marks, class_names)

        # Normalize the confusion matrix
        cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

        # Use white text if squares are dark; otherwise black.
        threshold = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            color = "white" if cm[i, j] > threshold else "black"
            plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label') 
        return figure

    def plot_to_image(figure):
    
        #save the plot to a png in memory
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
    
        #closing the figure to prevent from displaying in the notebook
        plt.close(figure)
    
        buf.seek(0)
    
        image = tf.image.decode_png(buf.getvalue(), channels=4)
    
        #add the batch dimension
        image = tf.expand_dims(image, 0)
    
        return image

    #file writer for logging cofusion matrix
    file_writer_cm = tf.summary.create_file_writer(log_dir + '/cm')
    
    def log_confusion_matrix(epoch, logs):
    #predict values for validation set
        test_pred_r = model.predict(val_images)
        test_pred = np.argmax(test_pred_r, axis=1)
    
        cm = sklearn.metrics.confusion_matrix(val_labels, test_pred)
    
        figure = plot_confusion_matrix(cm, class_names=['Trousers Male', 'Jeans Male', 'Trousers Female', 'Jeans Female'])
        cm_image = plot_to_image(figure)
    
        #log the confusion matrix as image summary
        with file_writer_cm.as_default():
            tf.summary.image('Confusion Matrix', cm_image, step=epoch)
            
    
    
    #defining callbacks
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq = 1, profile_batch = 0)
    cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end= log_confusion_matrix)
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        mode = 'auto',
        verbose=0,
        patience = 2,
        min_delta = 0,
        restore_best_weights= True
    )

    
    #training the model
    model.fit(
        train_images,
        train_labels,
        validation_data= (val_images,val_labels),
        verbose=2,
        epochs= NUM_EPOCHS,
        batch_size = BATCH_SIZE,
        callbacks=[tensorboard_callback,cm_callback,early_stopping]
        
    )
    
    #model performance evaluation
    _, accuracy = model.evaluate(val_images,val_labels)
    
    #saving the model
    model.save(r'saved Models\Model 3\Run-{}'.format(session_num))
    
    return accuracy


In [15]:
#logging the result function
def run(log_dir, hparams, session_num):
    
    with tf.summary.create_file_writer(log_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams, session_num)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)


In [16]:
session_num = 1

for filter_size in HP_FILTER_SIZE.domain.values:
    for num_filter in HP_NUM_FILTER.domain.values:
        for filter_size_2 in HP_FILTER_SIZE_2.domain.values:
        
            hparams = {
                HP_FILTER_SIZE : filter_size,
                HP_NUM_FILTER : num_filter,
                HP_FILTER_SIZE_2: filter_size_2
            }
        
            run_name = 'run-%d' %session_num
            print('---- Starting trial : %s' % run_name)
            print({h.name: hparams[h] for h in hparams})
            run('Logs/Model 3/hparam_tuning/' +run_name, hparams, session_num)
        
            session_num += 1

---- Starting trial : run-1
{'filter_size': 3, 'filters_number': 32, 'filter_size_2': 3}
Epoch 1/15
63/63 - 17s - loss: 1.1167 - accuracy: 0.5115 - val_loss: 1.0418 - val_accuracy: 0.5820
Epoch 2/15
63/63 - 17s - loss: 0.8843 - accuracy: 0.6441 - val_loss: 0.9380 - val_accuracy: 0.6200
Epoch 3/15
63/63 - 17s - loss: 0.7898 - accuracy: 0.6703 - val_loss: 0.9334 - val_accuracy: 0.5940
Epoch 4/15
63/63 - 17s - loss: 0.7125 - accuracy: 0.7175 - val_loss: 0.9152 - val_accuracy: 0.6440
Epoch 5/15
63/63 - 17s - loss: 0.6465 - accuracy: 0.7448 - val_loss: 0.8223 - val_accuracy: 0.6760
Epoch 6/15
63/63 - 17s - loss: 0.6350 - accuracy: 0.7423 - val_loss: 0.7492 - val_accuracy: 0.7140
Epoch 7/15
63/63 - 17s - loss: 0.5810 - accuracy: 0.7710 - val_loss: 0.7675 - val_accuracy: 0.7220
Epoch 8/15
63/63 - 18s - loss: 0.5542 - accuracy: 0.7880 - val_loss: 0.7414 - val_accuracy: 0.7240
Epoch 9/15
63/63 - 17s - loss: 0.5281 - accuracy: 0.7935 - val_loss: 0.7394 - val_accuracy: 0.7200
Epoch 10/15
63/63 - 

16/16 [==============================] - 3s 196ms/step - loss: 0.7858 - accuracy: 0.6780
INFO:tensorflow:Assets written to: saved Models\Model 3\Run-7\assets
---- Starting trial : run-8
{'filter_size': 3, 'filters_number': 128, 'filter_size_2': 5}
Epoch 1/15
63/63 - 167s - loss: 1.2691 - accuracy: 0.4900 - val_loss: 1.0764 - val_accuracy: 0.5560
Epoch 2/15
63/63 - 168s - loss: 0.9068 - accuracy: 0.6354 - val_loss: 0.9706 - val_accuracy: 0.6060
Epoch 3/15
63/63 - 167s - loss: 0.8254 - accuracy: 0.6653 - val_loss: 0.9463 - val_accuracy: 0.6080
Epoch 4/15
63/63 - 164s - loss: 0.7583 - accuracy: 0.6921 - val_loss: 0.8801 - val_accuracy: 0.6620
Epoch 5/15
63/63 - 166s - loss: 0.7262 - accuracy: 0.7165 - val_loss: 0.8622 - val_accuracy: 0.6420
Epoch 6/15
63/63 - 166s - loss: 0.6753 - accuracy: 0.7298 - val_loss: 0.8501 - val_accuracy: 0.6740
Epoch 7/15
63/63 - 166s - loss: 0.6585 - accuracy: 0.7405 - val_loss: 0.8005 - val_accuracy: 0.7240
Epoch 8/15
63/63 - 165s - loss: 0.5950 - accuracy: 0

63/63 - 117s - loss: 0.7223 - accuracy: 0.7033 - val_loss: 0.9110 - val_accuracy: 0.6660
Epoch 5/15
63/63 - 120s - loss: 0.6937 - accuracy: 0.7185 - val_loss: 0.9850 - val_accuracy: 0.6360
16/16 [==============================] - 3s 204ms/step - loss: 0.8751 - accuracy: 0.6620
INFO:tensorflow:Assets written to: saved Models\Model 3\Run-14\assets
---- Starting trial : run-15
{'filter_size': 5, 'filters_number': 128, 'filter_size_2': 3}
Epoch 1/15
63/63 - 111s - loss: 1.2667 - accuracy: 0.4648 - val_loss: 1.0805 - val_accuracy: 0.5320
Epoch 2/15
63/63 - 110s - loss: 0.8782 - accuracy: 0.6459 - val_loss: 0.9952 - val_accuracy: 0.6020
Epoch 3/15
63/63 - 109s - loss: 0.7891 - accuracy: 0.6858 - val_loss: 0.8808 - val_accuracy: 0.6580
Epoch 4/15
63/63 - 110s - loss: 0.7191 - accuracy: 0.7048 - val_loss: 0.8273 - val_accuracy: 0.6740
Epoch 5/15
63/63 - 110s - loss: 0.6626 - accuracy: 0.7325 - val_loss: 0.9076 - val_accuracy: 0.6400
Epoch 6/15
63/63 - 109s - loss: 0.6319 - accuracy: 0.7438 - v

Epoch 4/15
63/63 - 70s - loss: 0.7885 - accuracy: 0.6848 - val_loss: 0.8751 - val_accuracy: 0.6520
Epoch 5/15
63/63 - 69s - loss: 0.7578 - accuracy: 0.6936 - val_loss: 0.8551 - val_accuracy: 0.6480
Epoch 6/15
63/63 - 69s - loss: 0.7006 - accuracy: 0.7178 - val_loss: 0.8120 - val_accuracy: 0.6700
Epoch 7/15
63/63 - 70s - loss: 0.6742 - accuracy: 0.7395 - val_loss: 0.8014 - val_accuracy: 0.6780
Epoch 8/15
63/63 - 69s - loss: 0.6285 - accuracy: 0.7485 - val_loss: 0.7784 - val_accuracy: 0.6880
Epoch 9/15
63/63 - 69s - loss: 0.5927 - accuracy: 0.7580 - val_loss: 0.7875 - val_accuracy: 0.6920
Epoch 10/15
63/63 - 69s - loss: 0.5831 - accuracy: 0.7607 - val_loss: 0.7651 - val_accuracy: 0.7280
Epoch 11/15
63/63 - 69s - loss: 0.5502 - accuracy: 0.7855 - val_loss: 0.7462 - val_accuracy: 0.7080
Epoch 12/15
63/63 - 69s - loss: 0.5484 - accuracy: 0.7845 - val_loss: 0.8074 - val_accuracy: 0.7140
Epoch 13/15
63/63 - 69s - loss: 0.5058 - accuracy: 0.8029 - val_loss: 0.7423 - val_accuracy: 0.7440
Epoch 

In [17]:
%load_ext tensorboard
%tensorboard --logdir "Logs/Model 3/hparam_tuning"

In [18]:
%load_ext tensorboard
%tensorboard --logdir "Logs/Model 3/fit"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
